In [ ]:
import pandas as pd

In [2]:
dt = pd.read_csv('../data/recorridos-realizados-2019.csv',nrows=1000)


In [6]:
dt.id_estacion_origen.str.replace('_','.')

0      168.0
1       47.0
2      144.0
3      144.0
4       16.0
       ...  
995    191.0
996    106.0
997    186.0
998    186.0
999    121.0
Name: id_estacion_origen, Length: 1000, dtype: object

In [ ]:
#leemos los datos
#dt = pd.read_csv('../data/recorridos-realizados-2019.csv',nrows=1000)
dt = pd.read_csv('../data/recorridos-realizados-2019.csv')
dt['id_estacion_origen'] = pd.to_numeric(dt.id_estacion_origen\
                                     .str.replace('_','.'))
dt['id_estacion_destino'] = pd.to_numeric(dt.id_estacion_destino\
                                     .str.replace('_','.'))
dt.head()

In [ ]:
#obtenemos una tabla con cantidad de viajes por cada estacion de origen
viajesXestacion = dt.reindex(columns = ['id_estacion_origen','id_usuario'])\
                    .groupby('id_estacion_origen').count().reset_index()
viajesXestacion.columns = ['id_estacion_origen','n']
viajesXestacion.sort_values(by='n',ascending=False).head(3)

In [ ]:
#leemos el shape con los objetos geometricos de nuestras estaciones
estaciones = pd.read_csv('../carto/estaciones-bicicletas.csv')
estaciones = estaciones.reindex(columns = ['lat','long','nombre_estacion','id_estacion'])
estaciones.head()

In [ ]:
#importamos la funcion para crear puntos de la libreria de shapely
from shapely.geometry import Point


In [ ]:
#definimos una funcion que cree un Shapely Point a partir de un par de latlong
def crear_punto(lat,long):
    #noten como se invierte el orden tradicional de latlong, para ser long lat similar al orden matematico x,y
    return Point(long,lat)

In [ ]:
#probamos la funcion
crear_punto(long = estaciones.iloc[0].long,
            lat = estaciones.iloc[0].lat)

In [ ]:
#iteramos a traves de todo nuestro dataset de estacoines y guardamos en una lista sus pares latlong como Points
lista = []
for i in estaciones.index:
    punto = crear_punto(long = estaciones.loc[i].long,
            lat = estaciones.loc[i].lat)
    lista.append(punto)
lista

In [ ]:
#importamos geopandas
import geopandas as gpd

In [ ]:
#convertimos nuestro DataFrame de estaciones en un GeoDataFrame
crs = {'init' :'epsg:4326'}
gdf = gpd.GeoDataFrame(estaciones,
                       geometry=lista,
                       crs={'init' :'epsg:4326'})
gdf.head()


In [ ]:
type(gdf)

In [ ]:
#importamos las librerias de visualizacion
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#visualizamos nuestras estaciones
gdf.plot()

In [ ]:
#vemos que porcentajes de nuestras estaciones con viajes esta en nuestro shapefile
viajesXestacion.id_estacion_origen.isin(estaciones.id_estacion).sum() / len(viajesXestacion)

In [ ]:
#hacemos el merge en base a las columnas que actuan como clave
gdf = gdf.merge(viajesXestacion,
                              left_on = 'id_estacion',
                              right_on = 'id_estacion_origen',
                              how = 'inner'
                             )

In [ ]:
#creamos un plot que muestre el origen mas popular
fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(1,1,1)
gdf.plot(ax=ax,column='n',
                markersize=gdf['n']/100,
                alpha=0.5,legend=True) 
ax.set_axis_off()
plt.axis('equal')
ax.set_title('Origen de viajes en Ecobici - 2019');

In [ ]:
#creamos un viaje agrupando por origen como destino, lo que nos da el par Origen-Destino de un viaje
viajes = dt.reindex(columns = ['id_estacion_origen','id_estacion_destino','id_usuario'])\
                    .groupby(['id_estacion_origen','id_estacion_destino']).count().reset_index()
viajes.columns = ['id_estacion_origen','id_estacion_destino','n']
viajes.sort_values(by='n',ascending=False).head(3)

In [ ]:
#usamos como indice el indice verdadero
estaciones.index = estaciones.id_estacion
estaciones.head()

In [ ]:
#importamos la funcion LineString
from shapely.geometry import LineString

In [ ]:
#nos quedamos solo con los viajes cuyas estacinoes tenemos en el shape
viajes = viajes.loc[viajes.id_estacion_origen.isin(estaciones.index) &
                    (viajes.id_estacion_destino.isin(estaciones.index)),:]

In [ ]:
#producimos una linea con el origen destino
geometrias = []
for indice,fila in viajes.iterrows():
    
    id_origen = fila.id_estacion_origen
    id_destino = fila.id_estacion_destino 
    
    punto_origen = [estaciones.loc[id_origen,'long'],\
                    estaciones.loc[id_origen,'lat']]
    punto_destino = [estaciones.loc[id_destino,'long'],\
                     estaciones.loc[id_destino,'lat']]
    
    linea = LineString([punto_origen,punto_destino])
    
    geometrias.append(linea)

In [ ]:
#convertimos nuestro dataFrame en un geoDataFrame
viajes = gpd.GeoDataFrame(viajes,
                          geometry  = geometrias,
                          crs={'init' :'epsg:4326'})
viajes.head()

In [ ]:
estaciones=gpd.GeoDataFrame(estaciones,geometry='geometry',crs={'init' :'epsg:4326'})

In [ ]:
comunas = gpd.read_file('../carto/CABA_comunas.geojson')
comunas.crs

In [ ]:
fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(1,1,1)

comunas.plot(ax=ax,alpha=0.3)
estaciones.plot(ax=ax)
viajes.plot(
        ax=ax,
        column = viajes['n'],
        linewidth=viajes['n']/40,
        alpha=0.3,
        cmap='Blues',    
        scheme='boxplot',
        legend=True) 
    
ax.set_axis_off()
plt.axis('equal')
ax.set_title('Viajes en Ecobici - 2019');

In [ ]:
viajes.to_file('../carto/ecobici')